In [ ]:
#CURRENT PROJECT!!! 02/07/21 - user input, saved to a database using tkinter, python and sqlite3 

#updates include - amend booking function, cancel booking function, ensure all fields are completed prior to registering or making a booking, catches duplicate registers and duplicate bookings


#import the necessary libraries 
import sqlite3
import os, sys
import tkinter as tk
from tkinter import * 

#set up database filepath
DATABASE_FILEPATH='test_bookings.db'

if os.path.exists(DATABASE_FILEPATH):
        os.remove(DATABASE_FILEPATH)
    
conn=sqlite3.connect(DATABASE_FILEPATH)
cur=conn.cursor()

cur.executescript("""

CREATE TABLE Users(
user_id INTEGER PRIMARY KEY, 
name VARCHAR(200) NOT NULL, 
flat INTEGER NOT NULL, 
UNIQUE (name, flat)
);

CREATE TABLE Bookings(
booking_id INTEGER PRIMARY KEY,  
name VARCHAR(200) NOT NULL, 
flat INTEGER NOT NULL,
booking_date DATE NOT NULL, 
start_time TIME NULL, 
end_time TIME NULL,
UNIQUE (booking_date, start_time, end_time)
)


""")   


#create function to register users
def register_user():
    #create a database or connect to one
    conn=sqlite3.connect(DATABASE_FILEPATH)
    #create a cursor 
    cur=conn.cursor()
    
    #if text box entries empty, print a statement
    if ent_name.get() == '' and ent_flat.get() == '':
        answer.config(text='Please enter a valid name and flat.')
    
    else: 
        try:
            int(ent_flat.get())
            answer.config(text='That is a valid flat number.')
        except ValueError: 
            answer.config(text='That is NOT a valid flat number.')
    
    #check if user is already registered
        cur.execute('''SELECT name, flat FROM Users WHERE name = :name OR flat =:flat''', 
                {
                'name':ent_name.get(),
                'flat':ent_flat.get(), 
                })
        result=cur.fetchone()
        if not result: 
            #insert user data entry into table
            cur.execute('''INSERT INTO Users (name, flat) VALUES(:name, :flat)''',
                    {
                        'name':ent_name.get(),
                        'flat':ent_flat.get()
                    })

            cur.execute('SELECT user_id FROM Users WHERE name=:name AND flat=:flat', 
                        {'name':ent_name.get(), 
                         'flat':ent_flat.get()})
            user_id=cur.fetchone()[0]
            
            answer.config(text='Thank you for registering!')

            #commit changes 
            conn.commit()
            #close connection 
            conn.close()

            #delete text boxes
            ent_name.delete(0,END)
            ent_flat.delete(0,END)

        else: 
            answer.config(text='You have previously registered as a user.')

    
    
#create function to make a booking 
def make_booking():
    #create a database or connect to one
    conn=sqlite3.connect(DATABASE_FILEPATH)
    #create a cursor 
    cur=conn.cursor()
    #ensure all fields are complete prior to making a booking 
    if ent_name.get() == '':
        answer.config(text='Please enter a name.')
    elif ent_flat.get() == '':
        answer.confifg(text='Please enter a flat number.')
    elif ent_booking_date.get() =='':
        answer.config(text='Please enter a booking date.')
    elif ent_start_time.get() == '':
        answer.config(text='Please enter a start time.')
    elif ent_end_time.get()=='':
        answer.config(text='Please enter an end time.')
    
    #if all fields are complete - insert details into bookings table 
    else: 
    #check if user is registered prior to making a booking 
        cur.execute('''SELECT name, flat FROM Users WHERE name = :name OR flat =:flat''', 
                {
                'name':ent_name.get(),
                'flat':ent_flat.get(), 
                })
        result=cur.fetchone()
        
    
    #if the user is regsitered, check availability and insert all details into bookings table 
        if result: 
    #check if booking date and time is available
            cur.execute('''SELECT booking_date, start_time FROM Bookings WHERE booking_date=:booking_date OR start_time=:start_time''',
                        {
                        'booking_date':ent_booking_date.get(),
                        'start_time':ent_start_time.get()
                        })
            result2=cur.fetchone()
        
    #if user is registered and booking is available insert into bookings table 
            if not result2: 

                cur.execute('''INSERT OR IGNORE INTO Bookings (name, flat, booking_date, start_time, end_time) VALUES(:name, :flat, :booking_date, :start_time, :end_time)''',
                            {
                            'name':ent_name.get(),
                            'flat':ent_flat.get(), 
                            'booking_date': ent_booking_date.get(),
                            'start_time':ent_start_time.get(),
                            'end_time':ent_end_time.get()         
                        })

                #commit changes 
                conn.commit()
                #close connection 
                conn.close()
                answer.config(text='Thank you for making a booking!')

                print('Thank you for making a booking', ent_name.get(),'.' 'Your booking is for,', ent_booking_date.get(), 'from,', ent_start_time.get(), 'till', ent_end_time.get(), '.')

                #delete text boxes
                ent_name.delete(0,END)
                ent_flat.delete(0,END)
                ent_booking_date.delete(0,END)
                ent_start_time.delete(0,END)
                ent_end_time.delete(0,END)
        #if booking is not avaialable, please choose another date and time 
            else:
                answer.config(text='The selected date and time is not currently available.Please select an alternative date and time.')
            
            
            
    #if user is NOT registered ask them to register. 
        else: 
            answer.config(text='Please register as a user prior to making a booking.')

    
#create function to print a statement and do something 
def print_and_do():
    print('Hello World')

    if ent_name.get() == "Anoushka":
        answer.config(text="This is Anoushka's booking.")
        
        
#create function to amend a booking 
def amend_booking():
     #create a database or connect to one
    conn=sqlite3.connect(DATABASE_FILEPATH)
    #create a cursor 
    cur=conn.cursor()
    
    cur.execute("""UPDATE Bookings 
    SET booking_date=:booking_date, start_time=:start_time, end_time=:end_time WHERE booking_id=:booking_id""", 
                {
                'booking_date': ent_booking_date.get(),
                'start_time':ent_start_time.get(),
                'end_time':ent_end_time.get(),
                'booking_id': ent_booking_id.get()
                })
    
    #commit changes 
    conn.commit()
    #close connection 
    conn.close()
    
    print('Thank you. Your booking has been amended to,', ent_booking_date.get(), 'from,', ent_start_time.get(), 'till', ent_end_time.get(), '.')

    
    #delete text boxes
    ent_booking_date.delete(0,END)
    ent_start_time.delete(0,END)
    ent_end_time.delete(0,END)
    ent_booking_id.delete(0,END)


#create function to cancel or delete a booking 
def cancel_booking():
     #create a database or connect to one
    conn=sqlite3.connect(DATABASE_FILEPATH)
    #create a cursor 
    cur=conn.cursor()
    
    cur.execute("""DELETE FROM Bookings 
    WHERE booking_id=:booking_id""", 
                {
                'booking_id': ent_booking_id.get()
                })
    
    #commit changes 
    conn.commit()
    #close connection 
    conn.close()
    
    answer.config(text='Thank you. Your booking has been cancelled.')
    
    #delete text boxes
    ent_booking_id.delete(0,END)
   
     
#create window
window=tk.Tk()
window.title('Welcome to the bookings page')

#create text boxes for user input
ent_name=tk.Entry(width=30)
ent_name.grid(row=0, column=1, padx=20)

ent_flat=tk.Entry(width=30)
ent_flat.grid(row=1, column=1, padx=20)

ent_booking_date=tk.Entry(width=30)
ent_booking_date.grid(row=2,column=1, padx=20)

ent_start_time=tk.Entry(width=30)
ent_start_time.grid(row=3,column=1, padx=20)

ent_end_time=tk.Entry(width=30)
ent_end_time.grid(row=4,column=1, padx=20)

ent_booking_id=tk.Entry(width=30)
ent_booking_id.grid(row=7, column=1, padx=20)

#create label widget for text boxes 
lbl_name=tk.Label(text='Name')
lbl_name.grid(row=0, column=0)

lbl_flat=tk.Label(text='Flat Number')
lbl_flat.grid(row=1, column=0)

lbl_booking_date=tk.Label(text='Booking Date')
lbl_booking_date.grid(row=2, column=0)

lbl_start_time=tk.Label(text='Start Time')
lbl_start_time.grid(row=3, column=0)

lbl_end_time=tk.Label(text='End Time')
lbl_end_time.grid(row=4, column=0)

lbl_booking_id=tk.Label(text='Booking ID')
lbl_booking_id.grid(row=7, column=0)


#create a register user button
reg_btn=tk.Button(text='Register', command=register_user)
reg_btn.grid(row=5, columnspan=2, padx=10, pady=10)


#create a make a booking button
booking_btn=tk.Button(text='Make a booking', command=make_booking)
booking_btn.grid(row=6, columnspan=2, padx=10, pady=10)

#create a print and do button
do_something_btn=tk.Button(text='Do Something',command=print_and_do)
do_something_btn.grid(row=9, columnspan=2, padx=10, pady=10)

#create amend booking button 
amend_booking_btn=tk.Button(text='Amend a booking', command=amend_booking)
amend_booking_btn.grid(row=8, columnspan=2, padx=10, pady=10)   

#create a cancel booking button 
cancel_booking_btn=tk.Button(text='Cancel a booking', command=cancel_booking)
cancel_booking_btn.grid(row=9, columnspan=2, padx=10, pady=10)

#create an answer in the GUI
answer=tk.Label(text='')
answer.grid(row=10, column=0)
    
cur.close()
conn.commit()
conn.close() 


window.mainloop() 